In [ ]:
### SAYFADAN VERİ ÇEKME

# Kullanılacak kütüphaneyi import etme
from selenium import webdriver

# Tarayıcıyı başlatma
browser = webdriver.Chrome()

# Hesap ayarlarda seçili özelliklerin kullanılabilmesi(filmlerin adının orjinal dilinde olması) için,
#sayfada hesap girişi yapılacak bilgiler
username = "***@gmail.com"
password = "***"

# Yüklenecek web sayfası
browser.get("https://www.imdb.com/")

# Hesap girişi yapılabilmesi için olan işlemler
browser.find_element("xpath", f'//*[@id="imdbHeader"]/div[2]/div[5]/a/span').click()
browser.find_element("xpath", f'//*[@id="signin-options"]/div/div[1]/a[1]').click()
username_input = browser.find_element("xpath", f'//*[@id="ap_email"]')
password_input = browser.find_element("xpath", f'//*[@id="ap_password"]')

username_input.send_keys("***@gmail.com")
password_input.send_keys("***")

browser.find_element("xpath", f'//*[@id="signInSubmit"]').click()

browser.refresh()

# IMDB sitesinde önerilen ilk 1000 film adresine gider
browser.get("https://www.imdb.com/search/title/?count=100&groups=top_1000&sort=user_rating") 

# Filmleri içerisinde tutacak dizin tanımlama
film_list = [] 

# Sayfa geçişleri için döngü elemanları
i = 1
j = 1
k = 1

# Her sayfada 100 tane film olan 10 sayfanın sayfalar arası geçiş kodu
while i <= 102:
    if (i % 100 == 1 and i != 1): 
        if j == 1:
            browser.find_element("xpath", f'//*[@id="main"]/div/div[4]/a').click()
            i = 1
            j += 1
            k += 1
        else:
            if k == 10:
                break
            else:
                browser.find_element("xpath", f'//*[@id="main"]/div/div[4]/a[2]').click()
                i = 1
                k += 1
    
    browser.implicitly_wait(0.5)
    
    # İstenilen film bilgilerini sayfada bulma
    try:

        titles = browser.find_elements("xpath", f'/html/body/div[2]/div/div[2]/div[3]/div[1]/div/div[3]/div/div[{i}]/div[3]/h3/a')
        imdbs = browser.find_elements("xpath", f'//*[@id="main"]/div/div[3]/div/div[{i}]/div[3]/div/div[1]/strong')
        typess = browser.find_elements("xpath", f'//*[@id="main"]/div/div[3]/div/div[{i}]/div[3]/p[1]/span[5]')
        years = browser.find_elements("xpath", f'//*[@id="main"]/div/div[3]/div/div[{i}]/div[3]/h3/span[2]')
        stars_dir = browser.find_elements("xpath", f'//*[@id="main"]/div/div[3]/div/div[{i}]/div[3]/p[3]')
        durations = browser.find_elements("xpath", f'//*[@id="main"]/div/div[3]/div/div[{i}]/div[3]/p[1]/span[3]')
        certificates = browser.find_elements("xpath", f'//*[@id="main"]/div/div[3]/div/div[{i}]/div[3]/p[1]/span[1]')
        descriptions = browser.find_elements("xpath", f'//*[@id="main"]/div/div[3]/div/div[{i}]/div[3]/p[2]')


        # İstenilen film bilgilerini zip dosyasına dönüştürme
        for title,imdb,typei,year,star_dir,duration,certificate,description,in zip(titles, imdbs, typess, years,
                                                                                        stars_dir,durations,certificates,descriptions):
            
            film_titles = title.text
            film_imdbs = imdb.text
            film_types = typei.text
            film_years = year.text
            film_stars_dir = star_dir.text
            film_durations = duration.text
            film_certificates = certificate.text
            film_descriptions = description.text
            film_list.append([film_titles,  film_imdbs, film_types, film_years, 
                              film_stars_dir,film_durations,film_certificates,film_descriptions])
            
        i += 1 
        
    except Exception as e:
        print(f"Hata oluştu: {e}")


### CSV DOSYASINA DÖNÜŞTÜRME

import csv

# Çıktıları istediğimiz kolon adları ile CSV dosyasına kaydetmek için
with open('film_list.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["title","imdb","type","year","dir_star","duration","certificate","description"])  # Kolon isimleri
    writer.writerows(film_list) # CSV dosyası adı



### VERİ ÖN İŞLEME VE VERİ GÖRSELLEŞTİRME 

import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
#jupyter notebook'a özel matplotlib
%matplotlib inline  

import warnings
warnings.filterwarnings("ignore")

# Dataframe'i okuyup, df'ye atayıp ilk 5 veriye bakıyoruz
df = pd.read_csv("film_list.csv")
df.head()

# df hakkında bilgi alma
df.info()

# df'nin herhangi bir kolonun boş satırları olup olmadığını kontrol
df.isna().sum()

##Tüm Kolonları Tek Tek Kontrol

##Year Kolonu Düzeltme

# year kolonundaki parantezleri yok etme
df.year = df.year.str.replace('(','')
df.year = df.year.str.replace(')','')
df.year = df.year.str.strip()

# en çok hangi yıllarda kaç film yapıldığını gösterim
plt.figure(figsize=(6,5))
sb.set(style="darkgrid")
sb.countplot(y="year", data= df, palette="cool", order= df['year'].value_counts().index[0:15])

##dir_star Kolonu Düzeltme

# dfde dir_star kolonunu director ve star kolonu olarak ayırma
df[['Director', 'Stars']] = df['dir_star'].str.split('|', 2, expand=True)

# Kolonlarda yarım kalan düzeltmeleri yapma
df.Director = df.Director.str.replace('Director:','')
df.Stars = df.Stars.str.replace('Stars:','')
df.Director = df.Director.str.strip()
df.Stars = df.Stars.str.strip()

# Kullanılmayack dir_star kolonunu silme
df.drop(['dir_star'],axis = 1, inplace = True)
df.head()

# Kaç farklı yönetmen var
df["Director"].nunique() 

# İlk 10 en çok film yapan yönetmen gösterim
df['Director'].value_counts().head(10)

# İlk 10 en çok film yapan yönetmen gösterim
plt.figure(figsize=(10,8))
sb.barplot(y= df.Director.value_counts()[:10].sort_values().index, x=df.Director.value_counts()[:10].sort_values(ascending=False).values);
plt.title('Director with most number of movies', color='purple', fontsize=18)
plt.xticks(df.Director.value_counts()[:10].sort_values().values);
plt.xlabel('Number of Movies Released');

##duration Kolonu Düzeltme

# duration kolonundan min'i silme
df.duration = df.duration.str.replace('min','')
df.duration = df.duration.str.strip()
df.head()

# Object olan kolonu integer' çevirme
df['duration']=df['duration'].astype(str).astype(int)
df['duration'].head()

# Ortalama süreleri gösterme
sb.set(style="darkgrid")
sb.kdeplot(data=df['duration'], shade=True)

# Film süresinin trendleri
duration_year = df.groupby(['year']).mean()
duration_year = duration_year.sort_index()

plt.figure(figsize=(15,6))
sb.lineplot(x=duration_year.index, y=duration_year.duration.values)
plt.box(on=None)
plt.xticks(rotation=90)
plt.ylabel('Movie duration in minutes')
plt.xlabel('Year of released')
plt.title("Trends of Movie's Duration over the Years", fontsize=20, color='Green')

df.head()

##type Kolonu Düzeltme

# type adını film camiasında kullanılan Genre'ye çeviriyorum
df = df.rename(columns={"type":"Genre"})
df.Genre.head()

# Yazılan ilk türü alma
df['Genre'] = df['Genre'].apply(lambda x: x.split(",")[0])  
df['Genre'].head()

# En popüler türlerin grafiksel gösterimi
plt.figure(figsize=(9,5))
sb.barplot(x= df.Genre.value_counts()[:10].sort_values().index, y=df.Genre.value_counts()[:10].sort_values().values,palette='GnBu')
plt.title('Most Popular Genre', color='Blue', fontsize=20)
plt.xticks(rotation=45, fontsize=9)
plt.yticks(rotation=-45, fontsize=7)
plt.yticks(df.Genre.value_counts()[:10].sort_values().values)
plt.xlabel('GENRES')
plt.ylabel('Number of contents')

df.head()

#certificate Kolonu Düzeltme

# Hangi certificate'den kaç tane olduğunu gösterir
df['certificate'].value_counts()

# certificate grafik gösterimi
plt.figure(figsize=(6, 4))
plt.xticks(rotation=60)
sb.set(style="darkgrid")
sb.countplot(x="certificate", data= df, palette="Set2")

df.head()

##imdb Kolonu Düzeltme

# imdb kolonundan "," virgülü silme
df.imdb = df.imdb.str.replace(',','')
df.imdb = df.imdb.str.strip()
df.head()

# Object olan imdb kolonunu integer'a çevirme
df['imdb']=df['imdb'].astype(str).astype(int)
df['imdb'].head()

##description Kolonu

# description kolonunun böyle kalmasını sitediğim için değiştirmiyorum
df['description'].head()

#CSV KAYDETME

#Son hazırladığımız dataframe'i "last_list" adında yeni bir CSV dosyasına kaydediyoruz
df.to_csv('last_list.csv', index=False)

### MAKİNE ÖĞRENMESİ 

df = pd.read_csv("last_list.csv")
df.head()

df.info()

df.isna().sum()

from sklearn.feature_extraction.text import TfidfVectorizer

# Tüm kelimeleri küçük harf yaparak verileri temizleme.
def clean_data(x):
        return str.lower(x.replace(" ", ""))

# Kullanacağımız özellik kolonları ekleme ekleme
features=['Genre','Director','Stars','description','title']
filters = df[features]

# Features içinde seçilen kolonlara clean_data işlemi uygulandı
for feature in features:
    filters[feature] = filters[feature].apply(clean_data)
    
filters.head()

# df'nin seçilen features sutünlarını kullanarak bir metin oluşturma fonksiyonu
def create_soup(x):
    return x['Director'] + ' ' + x['Stars'] + ' ' +x['Genre']+' '+ x['description']

# Her satıra create_soup fonksiyonunu uygulayarak soup adında yeni kolon oluşturma 
filters['soup'] = filters.apply(create_soup, axis=1)


# CountVectorizer'ı import edip ve sayım matrisini oluşturma
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(filters['soup'])

# count_matrix'e göre kosinüs benzerliği matrisini hesaplama
from sklearn.metrics.pairwise import cosine_similarity

cos_sim = cosine_similarity(count_matrix, count_matrix)

# Ana DataFrame'imizin dizinini sıfırlama ve daha önce olduğu gibi ters eşlemeyi oluşturma
filters=filters.reset_index()
indices = pd.Series(filters.index, index=filters['title'])

def get_recommendations(title, cos_sim=cos_sim):
    title=title.replace(' ','').lower()
    idx = indices[title]
    
    try:
        idx = indices[title]
    except KeyError:
        return None  # Eğer film bulunamazsa None döndür

    # Bu filmle tüm filmlerin çift benzerlik puanlarını alma
    sim_scores = list(enumerate(cos_sim[idx]))
    
    # Filmleri benzerlik puanlarına göre sıralama
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # En benzer 10 filmin puanlarını alın
    sim_scores = sim_scores[1:11]

    # Film indekslerini alma
    movie_indices = [i[0] for i in sim_scores]

    # En çok benzeyen 10 filmi döndürme
    return df['title'].iloc[movie_indices]

get_recommendations('Fight Club')

get_recommendations('The GodFather')

get_recommendations('3 Idiots')

get_recommendations('Inception')


### TKİNTER İLE ARAYÜZ OLUŞTURMA

import tkinter as tk
from PIL import Image, ImageTk
import requests
from selenium import webdriver

def get_recommendations(title, cos_sim):
    title = title.replace(' ', '').lower()
    try:
        idx = indices[title]
    except KeyError:
        return None  # Eğer film bulunamazsa None döndür

    # Bu filmle tüm filmlerin çift benzerlik puanlarını alma
    sim_scores = list(enumerate(cos_sim[idx]))

    # Filmleri benzerlik puanlarına göre sıralama
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # En benzer 10 filmin puanlarını alın
    sim_scores = sim_scores[1:11]

    # Film indekslerini alma
    movie_indices = [i[0] for i in sim_scores]

    # En çok benzeyen 10 filmi döndürme
    return df['title'].iloc[movie_indices]

def oneri_getir():
    # Boşlukları temziler
    film_adi = entry_film.get().strip()
    # Eğer film adı boş butona basılmışsa text kutusunun içini siler ve hata mesajı döndürür
    if not film_adi:  
        oneri_text.config(state=tk.NORMAL)
        oneri_text.delete("1.0", tk.END)
        oneri_text.insert(tk.END, "The name of the movie is blank. Please enter the name of a valid movie.")
        oneri_text.config(state=tk.DISABLED)
        return

    oneriler = get_recommendations(film_adi, cos_sim)

    oneri_text.config(state=tk.NORMAL)
    oneri_text.delete("1.0", tk.END)

    # Eğer yazılan film listede yoksa hata mesajı döndürür
    if oneriler is not None:
        for i, film in enumerate(oneriler, 1):
            oneri_text.insert(tk.END, f"{i}. {film}\n")
    else:
        oneri_text.insert(tk.END, "Movie not found. Please enter a correct movie listed.")

    oneri_text.config(state=tk.DISABLED)

    
# Aranacak filmin tam adı bilinmiyorsa internet sitesine girerek film adına bakılabilir ve öneri alınabilir    
def oneri_website():
    browser = webdriver.Chrome()
    # Hesap ayarlarda seçili özelliklerin kullanılabilmesi(filmlerin adının orjinal dilinde olması) için,
    #sayfada hesap girişi yapılacak bilgiler
    username = "rana.batu01@gmail.com"
    password = "imdb1234"

    # Yüklenecek web sayfası
    browser.get("https://www.imdb.com/")

    # Hesap girişi yapılabilmesi için olan işlemler
    browser.find_element("xpath", f'//*[@id="imdbHeader"]/div[2]/div[5]/a/span').click()
    browser.find_element("xpath", f'//*[@id="signin-options"]/div/div[1]/a[1]').click()
    username_input = browser.find_element("xpath", f'//*[@id="ap_email"]')
    password_input = browser.find_element("xpath", f'//*[@id="ap_password"]')

    username_input.send_keys("rana.batu01@gmail.com")
    password_input.send_keys("imdb1234")

    browser.find_element("xpath", f'//*[@id="signInSubmit"]').click()

    browser.refresh()

    # IMDB sitesinde önerilen ilk 1000 film adresine gider
    browser.get("https://www.imdb.com/search/title/?count=100&groups=top_1000&sort=user_rating") 
    browser.mainloop()    

    # Tarayıcıyı kapat
    browser.quit()

# Butonların üzerine gelindiğinde veya basıldığında oluşacak efekler bu fonksiyonlarda tanımlanmıştır
def oneri_butona_giris(event):
    button.config(bg="#8C8C8C", fg="white") #Fare üzerine geldiğinde oluşacak renk
    button.config(relief=tk.SUNKEN)  # Butonu bastırılmış gibi göster
    

def website_butona_giris(event):    
    button_site.config(bg="#8C8C8C", fg="white") #Fare üzerine geldiğinde oluşacak renk
    button_site.config(relief=tk.SUNKEN)  # Butonu bastırılmış gibi göster
    
def oneri_butondan_cikis(event):
    button.config(bg="#454545", fg="white") #Fare üzerinden çekildiğinde oluşacak renk
    button.config(relief=tk.RAISED)  #Gölge animasyonları
    

def website_butondan_cikis(event):
    button_site.config(bg="#4682B4", fg="white") #Fare üzerinden çekildiğinde oluşacak renk
    button_site.config(relief=tk.RAISED)  #Gölge animasyonları
    
    
# Pencere oluşturur
window = tk.Tk()
window.title("Film Recommendation System")

# Pencere boyutu ve konumu ayarlar
window.geometry("800x600")

# Arkaplan resmini yükler
img = Image.open("afiss1.png")
img = img.resize((window.winfo_screenwidth(), window.winfo_screenheight()), Image.ANTIALIAS)
arkaplan_resmi = ImageTk.PhotoImage(img)

# Arka plan resmini etiketle ve en arkaya yerleştir
arkaplan_etiket = tk.Label(window, image = arkaplan_resmi)
arkaplan_etiket.place(x=0, y=0, relwidth = 1, relheight = 1)

# Resim açma ve oluşturma
resim = Image.open("imdb2.png")
tk_resim = ImageTk.PhotoImage(resim)

# Resmi etiketleme
etiket_resim = tk.Label(window, image = tk_resim)
etiket_resim.image = tk_resim 
etiket_resim.pack()

# Film giriş kutusu başlığı oluşturma
label_film = tk.Label(window, text="ENTER THE NAME OF THE MOVIES", font=("Stencil", 20), bg = 'red', fg = 'white')
label_film.pack(pady=10)

# Film adı yazma kısmı
entry_film = tk.Entry(window, width=40, fg='black', font=("Microsoft JhengHei", 12), relief = tk.SOLID, bd=2, bg = 'white', insertwidth=1)
entry_film.insert(0, "Write Here")
entry_film.bind("<FocusIn>", lambda event: entry_film.delete(0, tk.END) if entry_film.get() == "Write Here" else None)
entry_film.bind("<FocusOut>", lambda event: entry_film.insert(0, "Write Here") if not entry_film.get() else None)
entry_film.pack(pady=10)

# Öner butonu oluşturma ve oneri getir fonksiyonunu kullanma
button = tk.Button(window, text="SEND SUGGESTIONS", font=("Arial Black",9), command=oneri_getir, bg='#454545', fg='white',relief=tk.RAISED)
button.pack(pady=5)
entry_film.bind("<Return>", lambda event: oneri_getir()) #enter tuşu da çalıştırır

# Buton çerçeve efekti için bir çerçeve oluşturma
button_ust_cerceve = tk.Frame(window, bg="black")
button_ust_cerceve.pack(fill="both")

# Öneri butonu animasyonlarına bağlanma
button.bind("<Enter>", oneri_butona_giris)
button.bind("<Leave>", oneri_butondan_cikis)

# Web sitesi önerisi için web sitesine yönlendiren butan oluşturma
button_site = tk.Button(window, text="Go to the website for recommended movie names", font=("Arial Black", 9), command=oneri_website, bg='#4682B4', fg='white', relief=tk.RAISED)
button_site.pack(pady=5)

# Web sitesi buton animasyonlarına bağlanma
button_site.bind("<Enter>", website_butona_giris)
button_site.bind("<Leave>", website_butondan_cikis)

# Öneri metin alanı
oneri_text = tk.Text(window, width=40, height=10, font=("Arial Black",12),  wrap=tk.WORD, bg='white')
oneri_text.config(state=tk.DISABLED)
oneri_text.pack(pady=10)

# Pencereyi göster
window.mainloop()
